In [2]:
import asyncio 
import requests
import pandas as pd
import json
from urllib.parse import urlencode, urlunparse

In [1]:
#Get the coordinates by city. Gorge du Verdon has been replaced by Castelanne and Ariege by Foix to get more precise data
def get_coordinate(city,country):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'city': city,
        'country': country,
        "adresstype": 'city',
        'format': 'json'
    }
    r=requests.get(url=url,params=params)
    lat = r.json()[0]['lat']
    lon = r.json()[0]['lon']
    if r.status_code == 200:
        return {
            'city' : city,
            'lat' : lat,
            'lon': lon
            }
    else:
        return r.status_code

In [3]:
#Open the txt files with the cities
with open('cities.txt','r') as fichier:
    cities = json.load(fichier)

#Create a list of dictionnaries with the coordinates
coordinates = []
for city in cities:
    r = get_coordinate(city,'France')
    if r ==403:
        print('Not authorized')
        break
    else:
        coordinates.append(r)

In [4]:
#Create the dataframe
df = pd.DataFrame(coordinates)

In [5]:
df['code_insee'] = ''

#Get the insee code for each city from the API based on the GPS localisation
for city in df['city'].unique():
    url = 'https://geo.api.gouv.fr/communes?'
    params = {
        'lat' : df[df['city'] == city]['lat'].iloc[0],
        'lon': df[df['city'] == city]['lon'].iloc[0]
    }

    r = requests.get(url=url,params=params)
    code_insee = r.json()[0]['code']
    df.loc[df['city']==city,'code_insee']=code_insee


In [6]:
#Save it
df.to_csv('coordinates.csv',header=True ,index=False)